### Preprocess
- Re-categorizing ascending to descending, or vice versa doesn't have any impact the model accuracy.
- Use one hot encoder to set 'sex'? Gender seem binary '1' or '0' -- Can be ignored since it is a binary class.
- cp should be descending
- slope should be descending Downslop (severe), Horizontal and upsloping
- Use one hot encoder to set 'thal'c-- Can be ignored for (or experiment later)
\
DON'T WASTE TIME!!

Method:
1. Test Standardization vs Scaling with base model
2. Try stratified for CV.

'Age', 'Gender', 'Chest Pain', 'Rest BP (Sys)', 'Cholesterol (mg/dl)',' Blood Sugar (mg/dl) ','Rest ECG', 'Max Heart (Exe)', 'Angina (Exe)', 'ST Depression (Exe)', 'ST Slope (Exe))', 'Major Vessels', 'Thalasemia', 'Disease'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [88]:
data = pd.read_csv('data/uci-heart-disease/cleveland-cleaned.csv')
data.head(3)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1


In [93]:
X = data.drop('target', axis=1)
y = data['target']

In [84]:
X.head(3)

array([[ 0.93618065, -2.24062879,  0.75038004, ...,  1.06896529,
        -0.72197605,  0.65587737],
       [ 1.3789285 ,  0.87388018,  1.59626645, ...,  0.38177332,
         2.47842525, -0.89422007],
       [ 1.3789285 ,  0.87388018, -0.65943064, ...,  1.32666228,
         1.41162482,  1.17257652],
       ...,
       [ 1.48961547,  0.87388018,  0.69398761, ...,  2.01385425,
         1.41162482,  1.17257652],
       [ 0.27205887,  0.87388018, -0.09550637, ...,  0.12407633,
         0.34482438,  1.17257652],
       [ 0.27205887, -1.20245913, -0.09550637, ..., -0.90671163,
         0.34482438, -0.89422007]])

In [94]:
from sklearn.feature_selection import SelectKBest, chi2
ch2c = SelectKBest(score_func=chi2, k=13)
ch2c.fit(X, y)
cols = ['Age', 'Gender', 'Chest Pain', 'Rest BP (Sys)', 'Cholesterol (mg/dl)',' Blood Sugar (mg/dl) ','Rest ECG', 'Max Heart (Exe)', 'Angina (Exe)', 'ST Depression (Exe)', 'ST Slope (Exe)', 'Major Vessels', 'Thalasemia']
feature_scores_pvalue = pd.DataFrame({"Score":ch2c.scores_, "P_Value":np.round(ch2c.pvalues_, 3)}, index=cols)
feature_scores_pvalue.nlargest(n=13, columns="Score")

,Score,P_Value
Max Heart (Exe),187.053104,0.000
Major Vessels,82.730613,0.000
ST Depression (Exe),68.570533,0.000
Thalasemia,65.221093,0.000
Angina (Exe),35.508090,0.000
Age,22.917697,0.000
Cholesterol (mg/dl),20.855084,0.000
Rest BP (Sys),16.707463,0.000
Chest Pain,14.591587,0.000
Rest ECG,8.134652,0.004


In [ ]:
X = ch2c.transform(X)
X

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X[0:5]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=0)

In [ ]:
X_test[0:5]

In [ ]:
y_test.head()

# Base model
- classification report
- confusion matrix

In [ ]:
svc= SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
acc=accuracy_score(y_test, y_pred)*100
print("SVM - Accuracy: {:.3f}.".format(acc))
print("\nClassification Report")
print(classification_report(y_test, y_pred))

In [ ]:
# There were 60 records in test with equal class (1 and 0).
# Meanwhile, the prediction was 24:36
pd.DataFrame(y_pred).value_counts()

- <b>Precision</b> - 92% (22/24) prediction was correct in overall disease prediction
- <b>Recall</b> - 73% (22/30) prediction for disease was correct

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
plt.figure(figsize=(5, 5))
sns.heatmap(cm, annot=True, linewidths=.5, square=True, cmap='Blues', cbar=False)
# plt.ylabel('Actual');
# plt.xlabel('Predicted');
plt.ylabel('Truth');
plt.xlabel('Predicted');
plt.title("Confusion Matrix for Predicting Heart Disease (True=1, False=0)", size=12);

### GridSerachCV

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from numpy import arange
grid_svc= SVC()

parameters = dict()
parameters['kernel'] = ['rbf', 'poly', 'linear', 'sigmoid']
parameters['C'] = arange(1, 10, 1)
parameters['gamma'] = ['scale', 'auto']
parameters['class_weight'] = ['dict', 'balanced']

## Building Grid Search algorithm with cross-validation and acc score.

# grid_search_svc = GridSearchCV(grid_svc, parameters, scoring='accuracy', cv=5, n_jobs=-1)
grid_search_svc = GridSearchCV(grid_svc, parameters, scoring='accuracy', cv=5)

grid_search_svc.fit(X_train,y_train)
best_parameters_svc = grid_search_svc.best_params_
best_score_svc = grid_search_svc.best_score_
print(best_parameters_svc)
print(best_score_svc)

y_pred = grid_search_svc.predict(X_test)

# Get the accuracy score
svc_acc=accuracy_score(y_test, y_pred)*100
svc_pre=precision_score(y_test, y_pred, average='micro')
svc_recall=recall_score(y_test, y_pred, average='micro')
svc_f1_=f1_score(y_test, y_pred, average='micro')

print("\nSVM - Accuracy: {:.3f}.".format(svc_acc))
print("SVM - Precision: {:.3f}.".format(svc_pre))
print("SVM - Recall: {:.3f}.".format(svc_recall))
print("SVM - F1_Score: {:.3f}.".format(svc_f1_))
print("\nClassification Report")
print(classification_report(y_test, y_pred))

### RandomSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from numpy import arange
rand_svc= SVC()

parameters = dict()
parameters['kernel'] = ['rbf', 'poly', 'linear', 'sigmoid']
parameters['C'] = arange(1, 10, 1)
parameters['gamma'] = ['scale', 'auto']
parameters['class_weight'] = ['dict', 'balanced']

## Building Grid Search algorithm with cross-validation and acc score.

rand_search_svc = RandomizedSearchCV(rand_svc, parameters, scoring='accuracy', cv=5)

rand_search_svc.fit(X_train,y_train)
best_parameters_svc = rand_search_svc.best_params_
best_score_svc = rand_search_svc.best_score_
print(best_parameters_svc)
print(best_score_svc)

y_pred = rand_search_svc.predict(X_test)

# Get the accuracy score
svc_acc=accuracy_score(y_test, y_pred)*100
svc_pre=precision_score(y_test, y_pred, average='micro')
svc_recall=recall_score(y_test, y_pred, average='micro')
svc_f1_=f1_score(y_test, y_pred, average='micro')

print("\nSVM - Accuracy: {:.3f}.".format(svc_acc))
print("SVM - Precision: {:.3f}.".format(svc_pre))
print("SVM - Recall: {:.3f}.".format(svc_recall))
print("SVM - F1_Score: {:.3f}.".format(svc_f1_))
print("\nClassification Report")
print(classification_report(y_test, y_pred))